In [2]:
import os
import torch
import transformers
from langchain_community.tools.tavily_search import TavilySearchResults

In [11]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [20]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

In [21]:
for d in docs_list[0]:
    print(d)

('id', None)
('metadata', {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'})
('page_content', '\n\n\n\n\n\nLLM Powered Autonomous Agents | Lil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Po

In [4]:
os.environ["TAVILY_API_KEY"] = "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"

In [5]:
web_search_tool = TavilySearchResults(k=3)

In [6]:
from langchain.schema import Document

In [7]:
def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

In [8]:
state = web_search({"question": "What stocks are doing good right now?", "documents": None})

---WEB SEARCH---


In [10]:
print(state["question"])
for doc in state["documents"]:
    print(doc)

What stocks are doing good right now?
page_content='12/05/2023
Apple broke out and closed back above a $3 trillion...
Today's Spotlight
New AI Stocks Infographic
IBD Digital Webinar Sale
Join MarketDiem for $20/Year
More News
Nvidia shares have raced ahead in the last year on the chipmaker's strength in AI applications. This Is The Ultimate Donald Trump Stock: Is DWAC A Buy?
Related news
Dow Jones Futures Rise With Big AI Event On Tap; Apple Breaks Out To $3 Trillion Valuation
12/05/2023
Apple broke out and closed back above a $3 trillion valuation. D.R. Horton has operations in 118 markets in 33 states across the U.S. It operates in 45 of the top 50 markets in the U.S..
Products include single-family detached homes and attached homes, such as town homes, duplexes and condominiums.
 BREAKING: Futures Rise, But AI News On Tap
These Are The 5 Best Stocks To Buy And Watch Now
Buying a stock is easy, but buying the right stock without a time-tested strategy is incredibly hard. Inflation re

In [2]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

KeyboardInterrupt: 

In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])